In [2]:
import pandas as pd
import pickle
import requests
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Locate assets from GitHub
data_url = "https://raw.githubusercontent.com/tomweinandy/AIinPowerBI/main/vending_revenue.csv"
algo_url = "https://raw.githubusercontent.com/tomweinandy/AIinPowerBI/main/vending_model.pickle"

# Load data
# df = pd.read_csv('vending_revenue.csv')
df = pd.read_csv(data_url)
df = df.dropna()
df = df.drop(columns=["WeekStartingSat"])

# # Load the saved model from the pickle file
# with open('vending_model.pickle', 'rb') as file:
#     model_info = pickle.load(file)

# Load the pickled deep learning algorithm
response = requests.get(algo_url)
model_info = pickle.loads(response.content)

# Reconstruct the model architecture
loaded_model = tf.keras.models.model_from_json(model_info["architecture"])

# Set the model weights
loaded_model.set_weights(model_info["weights"])

# Preprocess the data
label_encoder = LabelEncoder()
df["Location"] = label_encoder.fit_transform(df["Location"])  # Encode location labels

# Split the data into input features (X)
X = df.drop("Location", axis=1)

# Predict the labels
predictions = loaded_model.predict(X)

# Change predictions to integers between 0 and 4
encoded_labels = [int(min(max(p.round(0), 0), 4)) for p in predictions]

# Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(encoded_labels)

# Add the predicted labels to the DataFrame
df.insert(0, "Predicted_Location", decoded_labels)

# Decode Location
df["Location"] = label_encoder.inverse_transform(df["Location"])

# Print accuracy
accuracy = sum(df["Location"] == df["Predicted_Location"])/len(df)
print(f"\nAccuracy: {accuracy}")

# Print the DataFrame with the predictions
df[["Location", "Predicted_Location"]]
df

8/8 [==============================] - 0s 1ms/step

Accuracy: 0.9365079365079365


,Predicted_Location,Location,Total,Quantity,AvgPriceItemSold,Type_Drink,Type_Food,Category_Carbonated,Category_Non Carbonated,Category_Salty,...,HealthyMarketing_True,Payment_Cash,Payment_Credit,DayOfWeek_Sat,DayOfWeek_Sun,DayOfWeek_Mon,DayOfWeek_Tue,DayOfWeek_Wed,DayOfWeek_Thu,DayOfWeek_Fri
0,Mall 1,Factory,58.50,35.0,1.671429,0.228571,0.771429,0.171429,0.057143,0.342857,...,0.485714,0.942857,0.057143,0.000000,0.000,0.228571,0.342857,0.171429,0.142857,0.114286
2,Mall 1,Mall 1,23.00,9.0,2.555556,0.666667,0.333333,0.666667,0.000000,0.333333,...,0.444444,0.111111,0.888889,0.555556,0.000,0.111111,0.111111,0.111111,0.111111,0.000000
3,Mall 2,Mall 2,17.75,9.0,1.972222,0.625000,0.375000,0.125000,0.250000,0.250000,...,0.750000,0.875000,0.125000,0.000000,0.125,0.125000,0.250000,0.000000,0.250000,0.250000
4,Office,Office,26.75,13.0,2.057692,0.153846,0.846154,0.000000,0.153846,0.692308,...,0.846154,0.769231,0.230769,0.000000,0.000,0.384615,0.076923,0.153846,0.307692,0.076923
5,Factory,Factory,61.50,40.0,1.537500,0.425000,0.575000,0.425000,0.000000,0.400000,...,0.375000,0.850000,0.150000,0.050000,0.000,0.050000,0.100000,0.025000,0.275000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,Mall 1,Mall 1,46.50,26.0,1.788462,0.541667,0.458333,0.000000,0.250000,0.333333,...,0.541667,0.666667,0.333333,0.041667,0.000,0.541667,0.083333,0.166667,0.000000,0.166667
258,Office,Mall 2,30.00,16.0,1.875000,0.533333,0.466667,0.333333,0.200000,0.466667,...,0.666667,0.666667,0.333333,0.000000,0.000,0.466667,0.066667,0.466667,0.000000,0.000000
259,Office,Office,3.00,2.0,1.500000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000,0.000000,0.500000,0.000000,0.500000,0.000000
262,Office,Mall 1,3.00,2.0,1.500000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
